In [1]:
!pip install wrapt --upgrade --ignore-installed
!pip install tensorflow

  Running setup.py bdist_wheel for wrapt ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/54/f9/95/099544e9f879f719b14cf567fabb5aa7984263df0f025f3eef
Successfully built wrapt
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 109.2MB 499kB/s  eta 0:00:01  11% |███▋                            | 12.3MB 93.2MB/s eta 0:00:02
    100% |████████████████████████████████| 51kB 35.9MB/s ta 0:00:01
    100% |████████████████████████████████| 3.2MB 17.5MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 37.7MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 42.7MB/s ta 0:00:01
    100% |████████████████████████████████| 2.7MB 22.2MB/s eta 0:00:01
    100% |████████████████████████████████| 20.2MB 2.6MB/s  eta 0:00:01
    100% |████████████████████████████████| 491kB 51.6MB/s eta 0:00:01
    100% |████████████████████

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_treebank_pos_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping taggers/maxent_treebank_pos_tagger.zip.
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
# NLP
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

# Classification
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

# Regression Models
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

# TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Recall, Precision
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Testing and optimization
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics.regression import mean_absolute_error, mean_squared_error, r2_score

# import module
from src.pipeline import *

# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   
import pickle
import datetime as dt
import glob

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")


Success - the MySageMakerInstance is in the us-east-1 region. You will use the 811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [6]:
# read csv from s3 bucket
s3 = boto3.resource('s3')
bucket = 'fakenewscorpus'
key = 'data/news_cleaned_2018_02_13.csv'
df = pd.read_csv('s3://{}/{}'.format(bucket,key), engine = 'python', nrows = 5000000)

In [27]:
df.memory_usage()

Index                     80
Unnamed: 0          39999208
id                  39999208
domain              39999208
type                39999208
url                 39999208
content             39999208
scraped_at          39999208
inserted_at         39999208
updated_at          39999208
title               39999208
authors             39999208
keywords            39999208
meta_keywords       39999208
meta_description    39999208
tags                39999208
summary             39999208
source              39999208
dtype: int64

In [29]:
import io
pkl_buffer = io.BytesIO()
df_key = 'data/5M_df.pkl'
pkl_df = df.to_pickle(df_key)
s3.Object(bucket,df_key).put(Body=pkl_df)

ParamValidationError: Parameter validation failed:
Invalid type for parameter Body, value: None, type: <class 'NoneType'>, valid types: <class 'bytes'>, <class 'bytearray'>, file-like object

In [22]:
s3 = boto3.resource('s3')
bucket = 'fakenewscorpus'
key = 'data/5M_df.pkl'
# obj = s3.get_object(Bucket='bucket', Key='key')
df = pickle.loads(s3.Bucket(bucket).Object(key).get()['Body'].read())

In [6]:
def balance(df):
    n_pos = len(df[df['label']==1])
    n_neg = len(df[df['label']==0])
    return n_pos, n_neg

n_pos, n_neg = balance(df)

n_pos/len(df)

0.33273061724603664

In [7]:
sw = pd.read_csv('data/sw1k.csv')['term'].to_numpy()

sample = df.sample(100000,axis=0)

balance(sample)

(33258, 66742)

In [5]:
tokens = tokenize(sample['content'],sw)

sample['token']=tokens

X = tokens
y = sample['label']

X_train, X_test, y_train, y_test = train_test_split(X,y)

bow, tf, tfidf, cv, tv = vectorize(X_train,max_features=5000,ngram=1)

In [49]:
pd.DataFrame(tfidf).to_csv('data/sample_tfidf.csv')

In [9]:
tfidf = pd.read_csv('data/sample_tfidf.csv')

In [58]:
tfidf.shape

(7500, 5000)

In [60]:
MAX_SEQ_LENGHT = len(max(X_train, key=len))
MAX_SEQ_LENGHT 

5466

In [54]:
N_FEATURES = len(bow)
X_train_sequences = pad_sequences(X_train, maxlen=MAX_SEQ_LENGHT, value=N_FEATURES)

ValueError: invalid literal for int() with base 10: 'fsu'

In [21]:
t = Tokenizer(num_words = 500)
s = df['content']
t.fit_on_texts(s)
sq = t_temp.texts_to_sequences(r)
# print('sequences : ', sq,'\n')
# print('word_index : ',t_temp.word_index )

sequences :  [[22], [3], [2], [1], [], [3], [5], [12], [], [7], [14], [7], [3], [5], [5], [3], [], [2], [4], [4], [22], [], [3], [], [15], [4], [15], [], [3], [2], [], [2], [3], [17], [11], [4], [8], [], [3], [5], [12], [], [2], [4], [13], [], [], [16], [3], [8], [8], [3], [20], [3], [17], [], [20], [3], [7], [], [8], [1], [15], [4], [8], [2], [6], [5], [16], [], [2], [9], [1], [], [5], [1], [20], [7], [], [], [3], [5], [5], [4], [14], [5], [10], [6], [5], [16], [], [2], [9], [3], [2], [], [7], [20], [6], [18], [2], [], [9], [3], [7], [], [19], [1], [1], [5], [], [14], [5], [18], [4], [11], [11], [4], [20], [1], [12], [], [19], [17], [], [9], [1], [8], [], [1], [28], [], [19], [4], [17], [18], [8], [6], [1], [5], [12], [], [10], [3], [11], [21], [6], [5], [], [9], [3], [8], [8], [6], [7], [], [4], [5], [], [2], [20], [6], [2], [2], [1], [8], [], [], [], [8], [1], [15], [4], [8], [2], [7], [], [9], [3], [21], [1], [], [7], [14], [8], [18], [3], [10], [1], [12], [], [4], [21], [1], [8], 

In [5]:
df = pickle.load('data/5M_df.pkl')

TypeError: file must have 'read' and 'readline' attributes

In [8]:
df = clean(df)

In [18]:
X = df['content']
y = df['label']
t = Tokenizer(num_words=10000)
t.fit_on_texts(df['content'])
X = t.texts_to_sequences(df['content'])

In [19]:
X_matrix = pad_sequences(X, maxlen=10000)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_matrix,y)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [22]:
X_matrix.shape

(2421128, 10000)

In [23]:
lstm_model = Sequential()

lstm_model.add(LSTM(units=64, return_sequences=True, input_shape=(1,10000)))
lstm_model.add(LSTM(64),)
lstm_model.add(Dense(units=1, activation='sigmoid'))
 
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC','BinaryAccuracy','Recall','Precision'])
print(lstm_model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 1, 64)             2576640   
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 2,609,729
Trainable params: 2,609,729
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
lstm_model.fit(X_train[:-400000], y_train[:-400000], 
          epochs=50, batch_size=128, verbose=1,
          validation_data=(X_train[-400000:], y_train[-400000:]))

Train on 1415846 samples, validate on 400000 samples
Epoch 1/50
1415846/1415846 [==============================] - 150s 106us/sample - loss: 0.6035 - auc_4: 0.6600 - binary_accuracy: 0.6732 - recall_4: 0.1759 - precision_4: 0.5621 - val_loss: 0.5964 - val_auc_4: 0.6717 - val_binary_accuracy: 0.6779 - val_recall_4: 0.2350 - val_precision_4: 0.5639
Epoch 2/50
1415846/1415846 [==============================] - 146s 103us/sample - loss: 0.5947 - auc_4: 0.6724 - binary_accuracy: 0.6810 - recall_4: 0.1905 - precision_4: 0.5969 - val_loss: 0.5899 - val_auc_4: 0.6789 - val_binary_accuracy: 0.6840 - val_recall_4: 0.1965 - val_precision_4: 0.6108
Epoch 3/50
1415846/1415846 [==============================] - 146s 103us/sample - loss: 0.5913 - auc_4: 0.6759 - binary_accuracy: 0.6829 - recall_4: 0.1929 - precision_4: 0.6056 - val_loss: 0.5907 - val_auc_4: 0.6776 - val_binary_accuracy: 0.6815 - val_recall_4: 0.2088 - val_precision_4: 0.5902
Epoch 4/50
1415846/1415846 [==============================]

In [24]:
!mkdir -p saved_model
model.save('saved_model/sample_model')

In [21]:
model = load_model('saved_model/sample_model',compile = False)
tfidf = pd.read_csv('data/sample_tfidf.csv')
X_train = tfidf.set_ind
y_train = sample['label']

In [27]:
lstm_model.save('saved_model/lstm_model_5M')

In [29]:
lstm_model.evaluate(X_test, y_test, use_multiprocessing=True)

605282/605282 [==============================] - 49s 81us/sample - loss: 0.5805 - auc_4: 0.6856 - binary_accuracy: 0.6895 - recall_4: 0.1988 - precision_4: 0.6417


[0.5804980644907652, 0.6856179, 0.6894588, 0.19880907, 0.641698]

In [6]:
X_test = [' '.join(row) for row in X_test]

In [7]:
tfidf_test = tv.transform(X_test)

In [75]:
model.evaluate(tfidf_test, y_test)

2500/2500 [==============================] - 37s 15ms/sample - loss: 0.6377 - auc_14: 0.5000 - binary_accuracy: 0.6656


[0.637664122581482, 0.5, 0.6656]

In [16]:
model2 = Sequential()
 
model2.add(Dense(units=500, activation='relu', input_dim=5000))
model2.add(Dense(units=1, activation='sigmoid'))
 
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC','BinaryAccuracy','Recall','Precision'])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 500)               2500500   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 501       
Total params: 2,501,001
Trainable params: 2,501,001
Non-trainable params: 0
_________________________________________________________________


In [17]:
model2.fit(tfidf[:-10000], y_train[:-10000], 
          epochs=30, batch_size=128, verbose=1,
          validation_data=(tfidf[-10000:], y_train[-10000:]))

Train on 65000 samples, validate on 10000 samples
Epoch 1/30
65000/65000 [==============================] - 4s 56us/sample - loss: 0.3315 - auc_2: 0.9143 - binary_accuracy: 0.8586 - recall: 0.6843 - precision: 0.8681 - val_loss: 0.2754 - val_auc_2: 0.9386 - val_binary_accuracy: 0.8855 - val_recall: 0.7291 - val_precision: 0.9064
Epoch 2/30
65000/65000 [==============================] - 3s 51us/sample - loss: 0.2491 - auc_2: 0.9514 - binary_accuracy: 0.8976 - recall: 0.7859 - precision: 0.8975 - val_loss: 0.2642 - val_auc_2: 0.9423 - val_binary_accuracy: 0.8889 - val_recall: 0.7557 - val_precision: 0.8919
Epoch 3/30
65000/65000 [==============================] - 3s 52us/sample - loss: 0.2161 - auc_2: 0.9637 - binary_accuracy: 0.9110 - recall: 0.8133 - precision: 0.9133 - val_loss: 0.2657 - val_auc_2: 0.9431 - val_binary_accuracy: 0.8889 - val_recall: 0.7884 - val_precision: 0.8639
Epoch 4/30
65000/65000 [==============================] - 3s 53us/sample - loss: 0.1778 - auc_2: 0.9763 - b

In [18]:
model2.save('saved_model/sample_model2')

In [19]:
model2.evaluate(tfidf_test, y_test)

25000/25000 [==============================] - 1s 59us/sample - loss: 0.6468 - auc_2: 0.9164 - binary_accuracy: 0.8898 - recall: 0.8058 - precision: 0.8510


[0.6467978135927022, 0.9164012, 0.88976, 0.8058158, 0.8509572]

In [ ]:
f_score = 0.81